# Data Preprocessing - heatmap

In [1]:
import pandas as pd
import numpy as np
import math
from datetime import datetime,timedelta
from scipy.stats import entropy
from sklearn.preprocessing import MinMaxScaler

In [2]:
df = pd.read_csv('mc1-reports-data.csv')
df.head()

,time,sewer_and_water,power,roads_and_bridges,medical,buildings,shake_intensity,location
0,2020-04-08 17:50:00,10.0,6.0,10.0,3.0,8.0,NaN,1
1,2020-04-09 13:50:00,2.0,10.0,0.0,8.0,4.0,0.0,1
2,2020-04-09 00:20:00,7.0,10.0,10.0,9.0,10.0,0.0,1
3,2020-04-08 17:25:00,1.0,1.0,2.0,10.0,7.0,NaN,1
4,2020-04-08 02:50:00,9.0,7.0,1.0,6.0,9.0,NaN,1


In [3]:
feature_list = ["sewer_and_water","power", "roads_and_bridges", "medical", "buildings","shake_intensity"]
cols = ['sewer_and_water','power','roads_and_bridges','medical','buildings','shake_intensity','location'] 

In [5]:
for i in range(len(feature_list)):
    feature = feature_list[i]
    print("{} has NAN: {}".format(feature,df[feature].isnull().sum()))

sewer_and_water has NAN: 171
power has NAN: 0
roads_and_bridges has NAN: 0
medical has NAN: 47441
buildings has NAN: 170
shake_intensity has NAN: 12144


In [6]:
regionGroup = df.groupby("location") #pandas.core.groupby.generic.DataFrameGroupBy object

In [7]:
#print out for finding the missing data points
for region_num in range(1,20):
    region_df = regionGroup.get_group(region_num)
    print('in region {}:'.format(region_num))
    for i in range(len(feature_list)):
        feature = feature_list[i]
        print("{} has NAN: {}".format(feature,region_df[feature].isnull().sum()))
    print('-----------------------')

in region 1:
sewer_and_water has NAN: 0
power has NAN: 0
roads_and_bridges has NAN: 0
medical has NAN: 0
buildings has NAN: 0
shake_intensity has NAN: 889
-----------------------
in region 2:
sewer_and_water has NAN: 0
power has NAN: 0
roads_and_bridges has NAN: 0
medical has NAN: 5294
buildings has NAN: 0
shake_intensity has NAN: 974
-----------------------
in region 3:
sewer_and_water has NAN: 0
power has NAN: 0
roads_and_bridges has NAN: 0
medical has NAN: 0
buildings has NAN: 0
shake_intensity has NAN: 770
-----------------------
in region 4:
sewer_and_water has NAN: 0
power has NAN: 0
roads_and_bridges has NAN: 0
medical has NAN: 2867
buildings has NAN: 0
shake_intensity has NAN: 225
-----------------------
in region 5:
sewer_and_water has NAN: 0
power has NAN: 0
roads_and_bridges has NAN: 0
medical has NAN: 0
buildings has NAN: 0
shake_intensity has NAN: 374
-----------------------
in region 6:
sewer_and_water has NAN: 0
power has NAN: 0
roads_and_bridges has NAN: 0
medical has N

In [10]:
##create timelist: elements are timestamp
timelist = []
time_current = pd.to_datetime("2020-04-06 00:00:00")
time_end = pd.to_datetime("2020-04-11 00:01:00")
##(time_start - time_end).total_seconds() < 0
while (time_end -time_current).total_seconds() > 0:
    timelist.append(time_current)
    time_current += timedelta(minutes=30)

region_dict = {(num+1): None for num in range(19)}

In [221]:
# starttimelist = []
# time_current = pd.to_datetime("2020-04-06 00:00:00")
# last_starttime = pd.to_datetime("2020-04-10 14:00:00")
# while (time_current <= last_starttime):
#     starttimelist.append(time_current)
#     time_current += timedelta(minutes=30)


In [11]:
#function for coverting time to the last interval (00:14 -> 00:00 00:59->00:30)
def convert_time_interval(time):
    approx = round(time.minute//30.0) * 30
    time = time.replace(minute=0)
    time += timedelta(seconds=approx * 60)
    return time

def convert_time_region(region_df):
    for idx, row in region_df.iterrows():
        current_time = row['time']
        time_interval = convert_time_interval(current_time)
        region_df.loc[idx,'time'] = time_interval
    return region_df

#function for sorting timeline of a region
def sort_regioni(i):
    """
    parameter: int i is referred to the region No.
    """
    region = regionGroup.get_group(i)
    region['time'] = pd.to_datetime(region.time)
    region = region.sort_values(by='time').reset_index().drop(labels=['index'], axis=1)
    region = convert_time_region(region)
    return region
    

In [12]:
for i in range(1,20):
    region_dict[i] = sort_regioni(i)

/Users/zhangyao/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [14]:
heatmap_data_dict = {num: pd.DataFrame(columns=cols,index= timelist) for num in range(1,20)} ##each element should be a dataframe

for i in range(1,20):
    region_df = region_dict[i]
    current_df = heatmap_data_dict[i]
    timegroup_mean_df = region_df.groupby('time').mean()
    for index,row in timegroup_mean_df.iterrows():   ##intex now is timestamp object
        current_df.loc[index] = row
    current_df['location'] = current_df['location'].fillna((current_df['location'].mean()))
    current_df = current_df.fillna(-1).reset_index()
    heatmap_data_dict[i] = current_df

In [17]:
merge_i_df = pd.concat([heatmap_data_dict[i] for i in range(1,20)])
merge_i_df['location'] = merge_10h_i_df['location'].astype(int)

In [18]:
def changename_i(string):
    if string == 'location' or string == 'index':
        return string
    else:
        return string+'_i'
merge_i_df.rename(mapper=changename_i, axis=1, inplace=True)

In [19]:
merge_i_df['location'] = merge_i_df['location'].astype(int)
merge_i_df

,index,sewer_and_water_i,power_i,roads_and_bridges_i,medical_i,buildings_i,shake_intensity_i,location
0,2020-04-06 00:00:00,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,1
1,2020-04-06 00:30:00,2.000000,10.000000,3.000000,5.000000,5.000000,-1.0,1
2,2020-04-06 01:00:00,7.500000,3.500000,6.000000,5.000000,4.500000,1.0,1
3,2020-04-06 01:30:00,9.000000,10.000000,2.000000,4.000000,0.000000,-1.0,1
4,2020-04-06 02:00:00,4.833333,6.166667,4.333333,8.333333,6.833333,-1.0,1
...,...,...,...,...,...,...,...,...
236,2020-04-10 22:00:00,7.000000,6.000000,0.000000,-1.000000,10.000000,-1.0,19
237,2020-04-10 22:30:00,10.000000,10.000000,8.000000,-1.000000,5.000000,-1.0,19
238,2020-04-10 23:00:00,3.333333,3.333333,8.333333,-1.000000,4.333333,1.0,19
239,2020-04-10 23:30:00,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,19


### using std to calculate uncertainty

In [20]:
heatmap_data_dict_std = {num: pd.DataFrame(columns=cols,index= timelist) for num in range(1,20)} 
##each element should be a dataframe

for i in range(1,20):
    region_df = region_dict[i]
    current_df = heatmap_data_dict_std[i]
    timegroup_std_df = region_df.groupby('time').std()
    for index,row in timegroup_std_df.iterrows():   ##intex now is timestamp object
        current_df.loc[index] = row
   # print('hi')
    current_df['location'] = i
   # current_df = current_df.fillna(-1).reset_index()
    current_df = current_df.reset_index()
    heatmap_data_dict_std[i] = current_df

In [21]:
heatmap_data_dict_std[1]

,index,sewer_and_water,power,roads_and_bridges,medical,buildings,shake_intensity,location
0,2020-04-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,1
1,2020-04-06 00:30:00,NaN,NaN,NaN,NaN,NaN,NaN,1
2,2020-04-06 01:00:00,3.535534,4.949747,0.0,2.828427,2.12132,NaN,1
3,2020-04-06 01:30:00,NaN,NaN,NaN,NaN,NaN,NaN,1
4,2020-04-06 02:00:00,3.430258,4.445972,4.033196,1.632993,3.371449,NaN,1
...,...,...,...,...,...,...,...,...
236,2020-04-10 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,1
237,2020-04-10 22:30:00,1.414214,0.0,2.828427,3.535534,5.656854,NaN,1
238,2020-04-10 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,1
239,2020-04-10 23:30:00,NaN,NaN,NaN,NaN,NaN,NaN,1


In [22]:

merge_u_df = pd.concat([heatmap_data_dict_std[i] for i in range(1,20)])


In [23]:
stdGroup = merge_u_df.groupby('index')

In [24]:
std_dict = {time: None for time in timelist}
for time in timelist:
    stdgroup = stdGroup.get_group(time)
    stdgroup.set_index(['index'],inplace=True)
    scaler = MinMaxScaler()
    scaler.fit(stdgroup)
    new_stdgroup = scaler.transform(stdgroup)
    new_df = pd.DataFrame(data=new_stdgroup,columns=cols, index=[num for num in range(1,20)])
    new_df['location'] = [num for num in range(1,20)]
    new_df['index'] = time
    std_dict[time] = new_df


/Users/zhangyao/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:400: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/Users/zhangyao/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:401: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)
/Users/zhangyao/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:400: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/Users/zhangyao/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:401: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)
/Users/zhangyao/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:400: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/Users/zhangyao/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:401: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)
/Use

In [25]:
#std_dict
def changename_u(string):
    if string == 'location' or string == 'index':
        return string
    else:
        return string+'_u'
merge_u_new_df = pd.concat([std_dict[time] for time in timelist]).fillna(-1)
merge_u_new_df.rename(mapper=changename_u, axis=1, inplace=True)
merge_u_new_df.reset_index(inplace=True)
merge_u_new_df.drop(axis=1,labels='level_0',inplace=True)

In [26]:
merge_i_df.set_index(['index','location'],inplace=True)
merge_u_new_df.set_index(['index','location'],inplace=True)

In [29]:
merge_u_new_df.head(30)

sewer_and_water_u   power_u  \
index               location                                
2020-04-06 00:00:00 1                 -1.000000 -1.000000   
                    2                  0.697615  0.193374   
                    3                  0.302214  0.459905   
                    4                  0.535413  0.587949   
                    5                  0.300000  0.166667   
                    6                  0.657557  0.377664   
                    7                 -1.000000 -1.000000   
                    8                  0.438316  0.782150   
                    9                 -1.000000 -1.000000   
                    10                -1.000000 -1.000000   
                    11                -1.000000 -1.000000   
                    12                 1.000000  0.166667   
                    13                -1.000000 -1.000000   
                    14                -1.000000 -1.000000   
                    15                 0.475395  0.632639   
                    16                -1.000000 -1.000000   
                    17                 0.000000  0.500000   
                    18                 0.700000  0.000000   
                    19                 0.400000  1.000000   
2020-04-06 00:30:00 1                 -1.000000 -1.000000   
                    2                  0.557928  0.270929   
                    3                  0.990239  0.660900   
                    4                  0.000000  0.246141   
                    5                  1.000000  0.852318   
                    6                  0.208539  0.115603   
                    7                 -1.000000 -1.000000   
                    8                 -1.000000 -1.000000   
                    9                  0.361522  1.000000   
                    10                -1.000000 -1.000000   
                    11                -1.000000 -1.000000   

                              roads_and_bridges_u  medical_u  buildings_u  \
index               location                                                
2020-04-06 00:00:00 1                   -1.000000  -1.000000    -1.000000   
                    2                    0.261145  -1.000000     0.791104   
                    3                    0.382196   1.000000     0.653864   
                    4                    0.000000  -1.000000     0.508306   
                    5                    0.291533   0.000000     0.500000   
                    6                    0.745569   0.706584     0.748015   
                    7                   -1.000000  -1.000000    -1.000000   
                    8                    0.675589  -1.000000     0.299441   
                    9                   -1.000000  -1.000000    -1.000000   
                    10                  -1.000000  -1.000000    -1.000000   
                    11                  -1.000000  -1.000000    -1.000000   
                    12                   0.291533  -1.000000     0.000000   
                    13                  -1.000000  -1.000000    -1.000000   
                    14                  -1.000000  -1.000000    -1.000000   
                    15                   0.771456  -1.000000     0.016485   
                    16                  -1.000000  -1.000000    -1.000000   
                    17                   0.822883  -1.000000     0.500000   
                    18                   0.291533  -1.000000     0.500000   
                    19                   1.000000  -1.000000     1.000000   
2020-04-06 00:30:00 1                   -1.000000  -1.000000    -1.000000   
                    2                    0.376494  -1.000000     0.704066   
                    3                    0.754693   0.138919     0.024715   
                    4                    0.404879  -1.000000     0.000000   
                    5                    0.000000   0.680584     0.784604   
                    6                    0.317893   1.000000     0.845245  

In [30]:
merge_i_df

,,sewer_and_water_i,power_i,roads_and_bridges_i,medical_i,buildings_i,shake_intensity_i
index,location,,,,,,
2020-04-06 00:00:00,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0
2020-04-06 00:30:00,1,2.000000,10.000000,3.000000,5.000000,5.000000,-1.0
2020-04-06 01:00:00,1,7.500000,3.500000,6.000000,5.000000,4.500000,1.0
2020-04-06 01:30:00,1,9.000000,10.000000,2.000000,4.000000,0.000000,-1.0
2020-04-06 02:00:00,1,4.833333,6.166667,4.333333,8.333333,6.833333,-1.0
...,...,...,...,...,...,...,...
2020-04-10 22:00:00,19,7.000000,6.000000,0.000000,-1.000000,10.000000,-1.0
2020-04-10 22:30:00,19,10.000000,10.000000,8.000000,-1.000000,5.000000,-1.0
2020-04-10 23:00:00,19,3.333333,3.333333,8.333333,-1.000000,4.333333,1.0


In [31]:
merge_df = pd.concat([merge_i_df,merge_u_new_df],axis=1)
merge_df = merge_df.reset_index()
merge_df.head(30)

,index,location,sewer_and_water_i,power_i,roads_and_bridges_i,medical_i,buildings_i,shake_intensity_i,sewer_and_water_u,power_u,roads_and_bridges_u,medical_u,buildings_u,shake_intensity_u
0,2020-04-06 00:00:00,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
1,2020-04-06 00:00:00,2,4.333333,8.333333,3.000000,-1.000000,5.000000,0.000000,0.697615,0.193374,0.261145,-1.000000,0.791104,0.000000
2,2020-04-06 00:00:00,3,3.833333,5.333333,8.166667,5.833333,4.166667,1.000000,0.302214,0.459905,0.382196,1.000000,0.653864,-1.000000
3,2020-04-06 00:00:00,4,7.500000,3.250000,9.250000,-1.000000,7.000000,0.500000,0.535413,0.587949,0.000000,-1.000000,0.508306,1.000000
4,2020-04-06 00:00:00,5,4.500000,3.000000,1.500000,8.000000,6.500000,-1.000000,0.300000,0.166667,0.291533,0.000000,0.500000,-1.000000
5,2020-04-06 00:00:00,6,4.428571,5.000000,3.857143,5.857143,6.571429,1.000000,0.657557,0.377664,0.745569,0.706584,0.748015,-1.000000
6,2020-04-06 00:00:00,7,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
7,2020-04-06 00:00:00,8,4.833333,5.750000,5.583333,-1.000000,5.083333,0.000000,0.438316,0.782150,0.675589,-1.000000,0.299441,-1.000000
8,2020-04-06 00:00:00,9,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
9,2020-04-06 00:00:00,10,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000


In [33]:
merge_df.to_csv('merge_ui.csv')